In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point

from mpl_toolkits.axes_grid1 import make_axes_locatable
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.neighbors import DistanceMetric
from math import radians

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


from sklearn import svm
from sklearn import metrics


from sklearn.datasets import make_moons, make_blobs
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, classification_report, make_scorer, recall_score, f1_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

In [3]:
# Utilisation du DF data_features qui contient toute les données nettoyées et le climat koppen intégré

df = pd.read_csv('data_features.csv', sep=',', index_col=0) 
df.head()

,sunshine,windgustspeed,humidity3pm,pressure3pm,year,windspeed3pm,temp_amplitude,cloud3pm,rainfall,climat,...,cloud9am,winddir9am_sin,latitude,month,geography,raintoday,evaporation,windspeed9am,windgustdir_cos,raintomorrow
0,10.9,44.0,22.0,1007.1,2008,24.0,9.5,7.0,0.6,0,...,8.0,1.224647e-16,-36.065766,12,0,0,7.6,20.0,-1.000000,0
1,12.9,44.0,25.0,1007.8,2008,22.0,17.7,1.0,0.0,0,...,1.0,9.238795e-01,-36.065766,12,0,0,12.0,4.0,-0.923880,0
2,8.0,46.0,30.0,1008.7,2008,26.0,12.8,2.0,0.0,0,...,7.0,1.224647e-16,-36.065766,12,0,0,8.0,19.0,-0.923880,0
3,2.2,24.0,16.0,1012.8,2008,9.0,18.8,6.0,0.0,0,...,8.0,-7.071068e-01,-36.065766,12,0,0,10.0,11.0,0.707107,0
4,6.7,41.0,33.0,1006.0,2008,20.0,14.8,8.0,1.0,0,...,7.0,3.826834e-01,-36.065766,12,0,0,4.8,7.0,-1.000000,0


In [4]:
features = ["sunshine","humidity3pm","pressure3pm","windspeed3pm","temp_amplitude",
            "cloud3pm","rainfall","climat","winddir3pm_sin","cloud9am", "latitude", "geography", "raintoday", "evaporation", "windgustdir_cos"]

In [5]:
#On supprime toutes les observations avec des NA
df1 = df.dropna()

target = df1["raintomorrow"]
data = df1[features]

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72707 entries, 0 to 72706
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sunshine         72707 non-null  float64
 1   windgustspeed    72707 non-null  float64
 2   humidity3pm      72707 non-null  float64
 3   pressure3pm      72707 non-null  float64
 4   year             72707 non-null  int64  
 5   windspeed3pm     72707 non-null  float64
 6   temp_amplitude   72707 non-null  float64
 7   cloud3pm         72707 non-null  float64
 8   rainfall         72707 non-null  float64
 9   climat           72707 non-null  int64  
 10  humidity9am      72707 non-null  float64
 11  winddir3pm_sin   72707 non-null  float64
 12  longitude        72707 non-null  float64
 13  winddir9am_cos   72707 non-null  float64
 14  cloud9am         72707 non-null  float64
 15  winddir9am_sin   72707 non-null  float64
 16  latitude         72707 non-null  float64
 17  month       

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=123)

In [8]:

from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 1.0, prior='uniform')
}




In [9]:
# Définition du modèle à optimiser (Random Forest)
model = RandomForestClassifier()




In [10]:
# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)



BayesSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
              scoring='accuracy',
              search_spaces={'max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize')})

In [19]:
# Affichage des meilleurs hyperparamètres trouvés
print('Meilleurs hyperparamètres :', opt.best_params_)

# Évaluation du modèle avec les hyperparamètres trouvés
score = opt.score(X_test, y_test)
print('Score de l\'optimisation bayésienne : {:.2f}'.format(score))

Meilleurs hyperparamètres : OrderedDict([('max_depth', 14), ('max_features', 0.554638842021933), ('min_samples_leaf', 2), ('min_samples_split', 6), ('n_estimators', 596)])
Score de l'optimisation bayésienne : 0.93


In [21]:
# Instanciation du modèle avec les meilleurs hyperparamètres
best_model = RandomForestClassifier(
    max_depth=14,
    max_features=0.554638842021933,
    min_samples_leaf=2,
    min_samples_split=6,
    n_estimators=596
)

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Entraînement du modèle sur l'ensemble des données d'entraînement
best_model.fit(X_train, y_train)

# Évaluation du modèle sur l'ensemble des données de test
score = best_model.score(X_test, y_test)
print('Score du modèle : {:.2f}'.format(score))

Score du modèle : 0.88


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Instanciation du modèle avec les meilleurs hyperparamètres
best_model = RandomForestClassifier(
    max_depth=14,
    max_features=0.554638842021933,
    min_samples_leaf=2,
    min_samples_split=6,
    n_estimators=596
)

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Entraînement du modèle sur l'ensemble des données d'entraînement
best_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calcul du score F1 weighted
score = f1_score(y_test, y_pred, average='weighted')
print('F1 score weighted du modèle :', score)


F1 score weighted du modèle : 0.8542153409127251


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Instanciation du modèle avec les meilleurs hyperparamètres
best_model = RandomForestClassifier(
    max_depth=14,
    max_features=0.554638842021933,
    min_samples_leaf=2,
    min_samples_split=6,
    n_estimators=596
)

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Entraînement du modèle sur l'ensemble des données d'entraînement
best_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calcul de l'accuracy
score = accuracy_score(y_test, y_pred)
print('Accuracy du modèle :', score)


Accuracy du modèle : 0.8786274240132032


In [15]:
from sklearn.metrics import classification_report

# Prédiction sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Affichage du rapport de classification
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.89      0.97      0.93     12269
           1       0.71      0.38      0.50      2273

    accuracy                           0.88     14542
   macro avg       0.80      0.68      0.71     14542
weighted avg       0.87      0.88      0.86     14542



In [16]:
from sklearn.metrics import f1_score

# Calcul du F1 score
f1 = f1_score(y_test, y_pred)
print('F1 score du modèle :', f1)


F1 score du modèle : 0.49700769449985754


In [22]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'learning_rate': Real(0.01, 1.0, prior='log-uniform'),
    'num_leaves': Integer(20, 200),
    'min_child_samples': Integer(5, 50),
    'subsample': Real(0.1, 1.0, prior='uniform'),
    'colsample_bytree': Real(0.1, 1.0, prior='uniform')
}

# Définition du modèle à optimiser (LGBMClassifier)
model = LGBMClassifier()

# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)


BayesSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
              scoring='accuracy',
              search_spaces={'colsample_bytree': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='normalize'),
                             'max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'min_child_samples': Integer(low=5, high=50, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize'),
                             'num_leaves': Integer(low=20, high=200, prior='uniform', transform='normalize'),
                             'subsample': Real(low=0.1, high=1.0, prior='uniform', transform='normalize')})

In [23]:
# Affichage des meilleurs hyperparamètres trouvés
print('Meilleurs hyperparamètres :', opt.best_params_)

# Évaluation du modèle avec les hyperparamètres trouvés
score_1 = opt.score(X_test, y_test)
print('Score de l\'optimisation bayésienne : {:.2f}'.format(score_1))

Meilleurs hyperparamètres : OrderedDict([('colsample_bytree', 0.636109111276708), ('learning_rate', 0.011237223602724626), ('max_depth', 10), ('min_child_samples', 5), ('n_estimators', 673), ('num_leaves', 139), ('subsample', 0.4463998077167358)])
Score de l'optimisation bayésienne : 0.88


In [24]:
# Application des hyperparametres

from lightgbm import LGBMClassifier

# Définition du modèle avec les meilleurs hyperparamètres
model = LGBMClassifier(colsample_bytree=0.636109111276708,
                       learning_rate=0.011237223602724626,
                       max_depth=10,
                       min_child_samples=5,
                       n_estimators=673,
                       num_leaves=139,
                       subsample=0.4463998077167358)

# Entraînement du modèle avec l'ensemble d'entraînement
model.fit(X_train, y_train)

# Évaluation du modèle avec l'ensemble de test
score = model.score(X_test, y_test)
print('Score du modèle avec les meilleurs hyperparamètres : {:.2f}'.format(score))

Score du modèle avec les meilleurs hyperparamètres : 0.88


In [25]:
from sklearn.ensemble import ExtraTreesClassifier


# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 1.0, prior='uniform')
}

# Définition du modèle à optimiser (ExtraTreesClassifier)
model = ExtraTreesClassifier()

# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_iter=20, n_jobs=-1,
              scoring='accuracy',
              search_spaces={'max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize')})

In [26]:
# Affichage des meilleurs hyperparamètres trouvés
print('Meilleurs hyperparamètres :', opt.best_params_)

# Évaluation du modèle avec les hyperparamètres trouvés
score_2 = opt.score(X_test, y_test)
print('Score de l\'optimisation bayésienne : {:.2f}'.format(score_2))

Meilleurs hyperparamètres : OrderedDict([('max_depth', 15), ('max_features', 1.0), ('min_samples_leaf', 3), ('min_samples_split', 2), ('n_estimators', 1000)])
Score de l'optimisation bayésienne : 0.88


In [30]:
# Appliquer les meilleurs hyperparametres

# Définition du modèle à optimiser (ExtraTreesClassifier)
model = ExtraTreesClassifier(max_depth=15, max_features=1.0, min_samples_leaf=3, min_samples_split=2, n_estimators=1000)

# Entraînement du modèle avec les meilleurs hyperparamètres trouvés
model.fit(X_train, y_train)

# Évaluation du modèle avec les hyperparamètres trouvés
score_2 = model.score(X_test, y_test)
print('Score avec les meilleurs hyperparamètres : {:.2f}'.format(score_2))

Score avec les meilleurs hyperparamètres : 0.88


In [31]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 1.0, prior='uniform')
}

In [32]:
# Définition du modèle à optimiser (Random Forest)
model = RandomForestClassifier()

# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
              scoring='accuracy',
              search_spaces={'max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize')})

In [33]:
# Affichage des meilleurs hyperparamètres trouvés
print('Meilleurs hyperparamètres :', opt.best_params_)

# Évaluation du modèle avec les hyperparamètres trouvés
score_3 = opt.score(X_test, y_test)
print('Score de l\'optimisation bayésienne : {:.2f}'.format(score_3))

Meilleurs hyperparamètres : OrderedDict([('max_depth', 15), ('max_features', 0.7497463720669447), ('min_samples_leaf', 1), ('min_samples_split', 2), ('n_estimators', 100)])
Score de l'optimisation bayésienne : 0.88


In [ ]:
# Application des meilleurs hyperparametres

